## Vehicle Detection

The goals / steps of this project are the following:

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
* Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector.
* Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
* Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.

---

In [151]:
import os
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.feature import hog
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from scipy.ndimage.measurements import label

%matplotlib qt

## Frame extraction from video

In [ ]:
dirname = 'video_image'
if not os.path.exists(dirname):
    os.makedirs(dirname)

vidcap = cv2.VideoCapture('test_video.mp4')
images =[]
ret = True
i = 0
while(ret):
    ret, frame = vidcap.read()
    
    if(ret == False):
        break
    filename = 'frame' +str(i).zfill(4) +'.png'
    cv2.imwrite(os.path.join(dirname, filename), frame)
    i += 1
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
vidcap.release()

In [2]:
# save the test image name as list type

imagedir_lists = glob.glob('video_image/*.png')
imagedir_lists = (sorted(imagedir_lists))

items = os.listdir("video_image/")
items = (sorted(items))
imagename_list = []
for names in items:
    if names.endswith(".png"):
        imagename_list.append(names)

## Training Data Load

In [3]:

folder_vehicles = glob.glob('vehicles/*')
folder_nonvehicles = glob.glob('non-vehicles/*')

cars = []
notcars = []

# Read in our vehicles and non-vehicles
for i in range(len(folder_vehicles)):
    image_names = glob.glob(folder_vehicles[i] + '/*.png')    
    for fname in image_names:
        cars.append(fname)
        
for i in range(len(folder_nonvehicles)):
    image_names = glob.glob(folder_nonvehicles[i] + '/*.png')    
    for fname in image_names:
        notcars.append(fname)    


___

In [ ]:
%matplotlib inline
### make example image
fig = plt.figure(figsize = (10,5))
# Call our function with vis=True to see an image output
for i in range(0, 4):
    
    ind = np.random.randint(0, len(cars))
    # Read in the car image
    car_example = mpimg.imread(cars[ind])

    # Generate a random index to look at a notcar image
    ind = np.random.randint(0, len(notcars))
    # Read in the notcar image
    notcar_example = mpimg.imread(notcars[ind])
    
    plt.subplot(2,4,1+i)
    plt.imshow(car_example, cmap = 'gray')
    plt.title('Car')
    plt.axis('off')
    plt.subplot(2,4,5+i)
    plt.imshow(notcar_example, cmap = 'gray')
    plt.title('Not Car')
    plt.axis('off')
    
    
plt.savefig("examples/car_not_car.png")

## Histogram of Oriented Gradients (HOG) feature extraction

In [4]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features


In [5]:
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
# from sklearn.model_selection import train_test_split

def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)

def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features


# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    #list_features =[]
    #list_spatial_features = []    
    #list_hist_features = []
    #list_hog_features =[]
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        
        # Read in each one by one
        image = mpimg.imread(file)
        
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
            #list_spatial_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
            #list_hist_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
            #list_hog_features.append(hog_features)
        features.append(np.concatenate(file_features))
        
    return features
    #if(list_feature  == False):
        # Return list of feature vectors
        #return features
    #else:

        #list_features.append(np.concatenate(list_spatial_features))
        #list_features.append(np.concatenate(list_hist_features))
        #list_features.append(np.concatenate(list_hog_features))
        #return list_features




___

In [ ]:
%matplotlib inline
### make HOG example image
# Generate a random index to look at a car image
ind = np.random.randint(0, len(cars))
# Read in the car image
car_example = mpimg.imread(cars[ind])

# Generate a random index to look at a notcar image
ind = np.random.randint(0, len(notcars))
# Read in the notcar image
notcar_example = mpimg.imread(notcars[ind])

cvt_car = cv2.cvtColor(car_example, cv2.COLOR_RGB2YCrCb)
cvt_notcar = cv2.cvtColor(notcar_example, cv2.COLOR_RGB2YCrCb)
# Define HOG parameters
orient = 9
pix_per_cell = 8
cell_per_block = 2

fig = plt.figure(figsize = (10,7))
plt.subplot(241)
plt.imshow(car_example, cmap = 'gray')
plt.title('Example Car Image')
plt.subplot(245)
plt.imshow(notcar_example, cmap = 'gray')
plt.title('Example Not Car Image')
# Call our function with vis=True to see an image output
for ch in range(0, 3):
    
    features, hog_image_car = get_hog_features(cvt_car[:,:,ch], orient, 
                            pix_per_cell, cell_per_block, 
                            vis=True, feature_vec=False)
    features, hog_image_notcar = get_hog_features(cvt_notcar[:,:,ch], orient, 
                            pix_per_cell, cell_per_block, 
                            vis=True, feature_vec=False)
    
    plt.subplot(2,4,2+ch)
    plt.imshow(hog_image_car, cmap='gray')
    plt.title('HOG Channel ' + str(ch))
    
    plt.subplot(2,4,6+ch)
    plt.imshow(hog_image_notcar, cmap='gray')
    plt.title('HOG Channel ' + str(ch))
    plt.tight_layout()
    
plt.savefig("examples/HOG_example.png")

## Linear SVM classifier

In [6]:

# Reduce the sample size because HOG features are slow to compute [test ver.]
#sample_size = 500
#sample_cars = cars[0:sample_size]
#sample_notcars = notcars[0:sample_size]

### TODO: Tweak these parameters and see how the results change.
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [None, None] # Min and max in y to search in slide_window()

t=time.time()
car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

/Users/yujin/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


169.88 Seconds to extract HOG features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
23.0 Seconds to train SVC...
Test Accuracy of SVC =  0.9913
My SVC predicts:  [ 1.  1.  0.  1.  1.  1.  1.  0.  0.  0.]
For these 10 labels:  [ 1.  1.  0.  1.  1.  1.  1.  0.  0.  0.]
0.039 Seconds to predict 10 labels with SVC


## Sliding Window Implementation

In [7]:

# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, vis = False):
    
    bbox_list = []
    draw_img = np.copy(img)
    img = img.astype(np.float32)#/255
    
    img_tosearch = img[ystart:ystop,:,:]
    ctrans_tosearch = convert_color(img_tosearch, conv='RGB2YCrCb')
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
            
            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)
            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1)) 
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,1),6) 
                # Assuming each "box" takes the form ((x1, y1), (x2, y2))
                bbox_list.append([[xbox_left, ytop_draw+ystart], [xbox_left+win_draw,ytop_draw+win_draw+ystart]])
                
    if(vis == True):
        return draw_img
    else:
        return bbox_list

In [ ]:
ystart = 400
ystop = 656
scale = 1.5

ind = np.random.randint(0, len(imagedir_lists))
image = mpimg.imread(imagedir_lists[ind])

out_img = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, vis = True)

plt.imshow(out_img)

In [235]:

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels, bbox_lists):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))

        xlen = (np.max(nonzerox)-np.min(nonzerox))
        ylen =  np.max(nonzeroy) - np.min(nonzeroy)
        minDist = min(xlen, ylen)
        maxDist = max(xlen, ylen)
        
        imgWidth = np.shape(img)[2]
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,1), 6)
        #if(maxDist/minDist < 1.3 and (imgWidth*0.25 < np.min(nonzerox)) and (imgWidth*0.75 > np.max(nonzerox)):
            # Draw the box on the image
            #cv2.rectangle(img, bbox[0], bbox[1], (0,0,1), 6)
            
        #else:
            
            #cXlist =[]
            #cYlist =[]
            #for bbox_list in bbox_lists:
                
            #    for box in bbox_list:
                    
                    # Add += 1 for all pixels inside each bbox
                    # Assuming each "box" takes the form ((x1, y1), (x2, y2))
             #       cYPos = min(box[0][1], box[1][1]) +  np.abs(box[0][1]-box[1][1])
             #       cXPos = min(box[0][0] ,box[1][0]) +  np.abs(box[0][0] -box[1][0])


              #      if((bbox[0][0] < cXPos) and (cXPos < bbox[1][0] ) and((bbox[0][1] < cYPos)and(cYPos < bbox[1][1]))):
               #         cXlist.append(cXPos)
                #        cYlist.append(cYPos)
            #stdX = np.std(cXlist)
            #stdY = np.std(cYlist)
            #if((stdX < 25) and (stdY < 18) and (stdX) !=np.nan and (stdY) !=np.nan):
            #    cv2.rectangle(img, bbox[0], bbox[1], (0,0,1), 6)
            #print('label: ', car_number)
            #print('x std: ', np.std(cXlist))
            #print('y std: ', np.std(cYlist))
            #cv2.rectangle(img, bbox[0], bbox[1], (0,0,1), 6)
            
    # Return the image
    return img
           
def make_new_boxlist(img, labels, bbox_lists):
    # Iterate through all detected cars
    new_box_list =[]
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))

        xlen = (np.max(nonzerox)-np.min(nonzerox))
        ylen =  np.max(nonzeroy) - np.min(nonzeroy)
        minDist = min(xlen, ylen)
        maxDist = max(xlen, ylen)
        
        xDistThr = 90
        yDistThr = 60
        centerMargin = 30
        cXlist =[]
        cYlist =[]
        
        new_box_candi_list = []
        for bbox_list in bbox_lists:
            for box in bbox_list:
                # Add += 1 for all pixels inside each bbox
                # Assuming each "box" takes the form ((x1, y1), (x2, y2))
                cYPos = min(box[0][1], box[1][1]) +  np.abs(box[0][1]-box[1][1])
                cXPos = min(box[0][0] ,box[1][0]) +  np.abs(box[0][0] -box[1][0])
           
                if((bbox[0][0] < cXPos + centerMargin) and (cXPos - centerMargin < bbox[1][0] ) and((bbox[0][1] < cYPos + centerMargin)and(cYPos- centerMargin < bbox[1][1]))):
                    cXlist.append(cXPos)
                    cYlist.append(cYPos)
                    new_box_candi_list.append(box)
                    
        if(len(cXlist) <= 1 and len(cYlist) <= 1):
            for box in new_box_candi_list:
                new_box_list.append(box)
                
        else:   
            stdX = np.std(cXlist)
            stdY = np.std(cYlist)
            meanX = np.mean(cXlist)
            meanY = np.mean(cYlist)
            halfWidth = np.shape(image)[1]/2.0
            horThr =  35 + np.abs(halfWidth - meanX) * 50/halfWidth
            #verThr = 30 + np.abs(halfWidth - meanX) * 20/halfWidth
            i = 0
            if(((stdX < horThr) and (stdY < horThr))):                
                for box in new_box_candi_list:
                    condA = ((np.abs(cXlist[i] - meanX) < xDistThr) and (np.abs(cYlist[i] - meanY) < yDistThr))                    
                    i += 1
                    if(condA):
                        new_box_list.append(box)

            #print('label: ', car_number)
            #print('cxlist: ', cXlist)
            #print('cylist: ', cYlist)
            #print('x std: ', stdX, '/', horThr)
            #print('y std: ', stdY, '/', horThr)
            #print('x mean: ', meanX)
            #print('y mean: ', meanY)
        
            
        

    return new_box_list




AttributeError: module 'numpy' has no attribute 'inverse'

In [ ]:
%matplotlib qt
box_list = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)

heat = np.zeros_like(image[:,:,0]).astype(np.float)
# Add heat to each box in box list
heat = add_heat(heat,box_list)
    
# Apply threshold to help remove false positives
heat = apply_threshold(heat, 0)

# Visualize the heatmap when displaying    
heatmap = np.clip(heat, 0, 255)

# Find final boxes from heatmap using label function
labels = label(heatmap)
draw_img = draw_labeled_bboxes(np.copy(image), labels)

fig = plt.figure()
plt.subplot(121)
plt.imshow(draw_img)
plt.title('Car Positions')
plt.subplot(122)
plt.imshow(heatmap, cmap='hot')
#plt.colorbar()
plt.title('Heat Map')
fig.tight_layout()

In [ ]:
bbox_list = []
scale_list = [1.0, 1.5, 1.75, 2.0]
i = 0
fig = plt.figure(figsize = (30,5))
for scale in scale_list:
    out_img = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, vis = True)
    plt.subplot(1,4,i+1)
    plt.imshow(out_img)
    plt.axis('off')
    plt.title('Scale: ' + str(scale))
    fig.tight_layout()
    
    i += 1
plt.savefig("examples/sliding_window.png")

In [ ]:
%matplotlib inline

scale_list = [1, 1.5, 1.75, 2.0]
folder_testimg = glob.glob('test_images/*.jpg')
folder_testimg = (sorted(folder_testimg))
i=1
fig = plt.figure(figsize = (30, 10))
imglist =[]
for file in folder_testimg:
    image = mpimg.imread(file)
    image = image/255.0
    bbox_list = []
    for scale in scale_list:
        bboxes = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
        for bbox in bboxes:
            bbox_list.append(bbox)

    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Add heat to each box in box list
    heat = add_heat(heat,bbox_list)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat, 2)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    imglist.append(draw_img)
    
    plt.subplot(2, 3, i)
    plt.imshow(draw_img)
    plt.title('Example '+ str(i))
    i += 1
fig.tight_layout()
plt.savefig("examples/output_bboxes.png")

## Test Vedio Result

In [ ]:
scale_list = [1.5, 1.75, 2.0]

In [ ]:
i = 0
dirname = 'result_test_image'
if not os.path.exists(dirname):
    os.makedirs(dirname)
    
    
for file in image_lists[0:5]:
    image = mpimg.imread(file)
    
    bbox_list = []
    prev_heat = np.zeros_like(image[:,:,0]).astype(np.float)
    acc_heat = []
    for scale in scale_list:
        bboxes = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
        for bbox in bboxes:
            bbox_list.append(bbox)

    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Add heat to each box in box list
    heat = add_heat(heat,bbox_list)
    
    acc_heat = np.add(heat, prev_heat)
    prev_heat = heat
    # Apply threshold to help remove false positives
    heat = apply_threshold(acc_heat, 2)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)
     
    
    # Find final boxes from heatmap using label function
    labels = label(acc_heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    draw_img = draw_img*255
    result_bgr = cv2.cvtColor(draw_img, cv2.COLOR_RGB2BGR)
    
    filename = 'result' +str(i).zfill(4) +'.png'
    cv2.imwrite(os.path.join(dirname, filename), result_bgr)
    print(filename)
    i += 1



In [ ]:
prev_heat = np.ones_like(image[:,:,0]).astype(np.float)
prev_heat2 = np.ones_like(image[:,:,0]).astype(np.float)*2

a = np.add(prev_heat, prev_heat2)*0.5
print(np.sum(np.ravel(a)))
plt.imshow(a)
plt.colorbar()

In [ ]:
import os
from moviepy.editor import ImageSequenceClip

items = os.listdir("result_test_image/")
imagelist = []
for names in items:
    if names.endswith(".png"):
        imagelist.append("result_test_image/" + names)
imagelist = (sorted(imagelist))

clip = ImageSequenceClip(imagelist, fps = 30)
clip.write_videofile("result_test_video.mp4")

## Project video result

In [ ]:
dirname = 'project_video_image'
if not os.path.exists(dirname):
    os.makedirs(dirname)

vidcap = cv2.VideoCapture('project_video.mp4')
images =[]
ret = True
i = 0
while(ret):
    ret, frame = vidcap.read()
    
    if(ret == False):
        break
    filename = 'frame' +str(i).zfill(4) +'.png'
    cv2.imwrite(os.path.join(dirname, filename), frame)
    i += 1
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
vidcap.release()

projImage_lists = glob.glob('project_video_image/*.png')
projImage_lists = (sorted(projImage_lists))



In [9]:
projImage_lists = glob.glob('project_video_image/*.png')
projImage_lists = (sorted(projImage_lists))
image = mpimg.imread(projImage_lists[0]) 
np.shape(image)[0]
heat_queue = np.zeros([np.shape(image)[0], np.shape(image)[1], 10])

In [35]:
scale_list = [1.0, 1.5, 1.75, 2.0]
ystart = 400
ystop = 656
np.shape(image)[2]

3

In [236]:
i = 0
dirname = 'output_images_test'
if not os.path.exists(dirname):
    os.makedirs(dirname)
    
fig = plt.figure(figsize = (30, 10))

image = mpimg.imread(projImage_lists[0])   
prev_heat = np.zeros_like(image[:,:,0]).astype(np.float)
fa_num = 6
heat_queue = np.zeros([np.shape(image)[0], np.shape(image)[1], fa_num])
bbox_queue = list(np.zeros(fa_num))
mean_img_queue = np.zeros([np.shape(image)[0], np.shape(image)[1], np.shape(image)[2], fa_num])

#prev_heat = prev_heat * 2
q = 0
acc_heat = []
for file in projImage_lists: #1035:1050 #650:680 #1035:1080 #700:770
    #print('---', i , '---')
    image = mpimg.imread(file)    
    bbox_list = []     
        
    for scale in scale_list:
        bboxes = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
        for bbox in bboxes:
            bbox_list.append(bbox)

    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Add heat to each box in box list
    heat = add_heat(heat,bbox_list)
    heat = apply_threshold(heat, 1)

    #alpha = 0.8
    #if(q < 1):
        #acc_heat = heat.copy()
        #heat = apply_threshold(acc_heat, 0.3)
        
    #else:
        #acc_heat = np.multiply(alpha , acc_heat) + np.multiply(1-alpha , heat)
        #heat = apply_threshold(acc_heat, 0.3)
        
        
    if(q < fa_num):
        heat_queue[:,:,q] = heat
        acc_heat = np.mean(heat_queue[:,:,0:q+1], axis = 2)
        # Apply threshold to help remove false positives
        bbox_queue[q] = bbox_list
        q += 1
    else:
        tmp_queue = heat_queue[:,:,1:fa_num].copy()
        heat_queue[:,:,0:fa_num-1] = tmp_queue
        heat_queue[:,:,fa_num-1] = heat
        acc_heat = np.mean(heat_queue, axis = 2)
    
        # Apply threshold to help remove false positives        
        tmp_bbox = bbox_queue[1:fa_num].copy()
        bbox_queue[0:fa_num-1] = tmp_bbox
        bbox_queue[fa_num-1] = bbox_list
        
    heat = apply_threshold(acc_heat, 0.1)
    
    
    #plt.clf()
    #plt.subplot(121)
    #plt.imshow(heat)
    #plt.colorbar()
    #filename = 'accheatmap' +str(i).zfill(4) +'.png'
    #plt.savefig("output_images_test2/" + filename)   
    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)
    
    # Find final boxes from heatmap using label function
    labels = label(heatmap)

    new_box_list = make_new_boxlist(np.copy(image), labels, bbox_queue[0:q])
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    new_heat = add_heat(heat,new_box_list)
    #plt.subplot(122)
    #plt.imshow(new_heat)
    #plt.colorbar()
    #filename = 'newheatmap' +str(i).zfill(4) +'.png'
    #plt.savefig("test_heatmap/" + filename)  
    
    new_heat = apply_threshold(new_heat, (q))
    
     
    
    
    heatmap = np.clip(new_heat, 0, 255)
    new_labels = label(heatmap)
    
    #draw_img = draw_labeled_bboxes(np.copy(image), labels, bbox_queue[0:q])
    draw_img = draw_labeled_bboxes(np.copy(image), new_labels, new_box_list)
    draw_img = draw_img*255
    result_bgr = cv2.cvtColor(draw_img, cv2.COLOR_RGB2BGR)
    
    filename = 'result' +str(i).zfill(4) +'.png'
    cv2.imwrite(os.path.join(dirname, filename), result_bgr)
    
    
    
    
    
    
    
    i += 1




KeyboardInterrupt: 

In [137]:
scale_list   

[1.0, 1.5, 1.75, 2.0]

In [234]:
import os
from moviepy.editor import ImageSequenceClip

items = os.listdir("output_images_test/")
imagelist = []
for names in items:
    if names.endswith(".png"):
        imagelist.append("output_images_test/" + names)
imagelist = (sorted(imagelist))

clip = ImageSequenceClip(imagelist, fps = 30)
clip.write_videofile("result_project_video_test.mp4")

[MoviePy] >>>> Building video result_project_video_test.mp4
[MoviePy] Writing video result_project_video_test.mp4


 99%|█████████▉| 82/83 [00:06<00:00, 13.65it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: result_project_video_test.mp4 



In [102]:
## example image


num_image = len(projImage_lists)
rand_image = np.random.randint(num_image, size=6)
rand_image_list = projImage_lists[rand_image]  
    
i = 0
dirname = 'output_images_test2'
if not os.path.exists(dirname):
    os.makedirs(dirname)
    
fig = plt.figure(figsize = (30, 10))

image = mpimg.imread(projImage_lists[0])   
prev_heat = np.zeros_like(image[:,:,0]).astype(np.float)
fa_num = 8
heat_queue = np.zeros([np.shape(image)[0], np.shape(image)[1], fa_num])
bbox_queue = list(np.zeros(fa_num))
mean_img_queue = np.zeros([np.shape(image)[0], np.shape(image)[1], np.shape(image)[2], fa_num])

#prev_heat = prev_heat * 2
q = 0
acc_heat = []


ex_heatmap =[]
ex_bounding_boxes =[]
ex_label =[]
ex_out_bounding_boxes =[]
ex_images = []
for file in rand_image_list: #1035:1050
    #print('---', i , '---')
    image = mpimg.imread(file)  
    ex_images.append(image)
    bbox_list = []     
        
    for scale in scale_list:
        bboxes = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
        for bbox in bboxes:
            bbox_list.append(bbox)

    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Add heat to each box in box list
    heat = add_heat(heat,bbox_list)
    heat = apply_threshold(heat, 2)

    #alpha = 0.8
    #if(q < 1):
        #acc_heat = heat.copy()
        #heat = apply_threshold(acc_heat, 0.3)
        
    #else:
        #acc_heat = np.multiply(alpha , acc_heat) + np.multiply(1-alpha , heat)
        #heat = apply_threshold(acc_heat, 0.3)
        
        
    if(q < fa_num):
        heat_queue[:,:,q] = heat
        acc_heat = np.mean(heat_queue[:,:,0:q+1], axis = 2)
        # Apply threshold to help remove false positives
        bbox_queue[q] = bbox_list
        q += 1
    else:
        tmp_queue = heat_queue[:,:,1:fa_num].copy()
        heat_queue[:,:,0:fa_num-1] = tmp_queue
        heat_queue[:,:,fa_num-1] = heat
        acc_heat = np.mean(heat_queue, axis = 2)
    
        # Apply threshold to help remove false positives        
        tmp_bbox = bbox_queue[1:fa_num].copy()
        bbox_queue[0:fa_num-1] = tmp_bbox
        bbox_queue[fa_num-1] = bbox_list
        
    heat = apply_threshold(acc_heat, 0.3)
    
    
    #plt.clf()
    #plt.subplot(121)
    #plt.imshow(heat)
    #plt.colorbar()
    #filename = 'accheatmap' +str(i).zfill(4) +'.png'
    #plt.savefig("output_images_test2/" + filename)   
    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)
    
    # Find final boxes from heatmap using label function
    labels = label(heatmap)

    new_box_list = make_new_boxlist(np.copy(image), labels, bbox_queue[0:q])
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    new_heat = add_heat(heat,new_box_list)
    ex_heatmap.append(new_heat)
    new_heat = apply_threshold(new_heat, (q-1)*2.5)
    
    #plt.subplot(122)
    #plt.imshow(new_heat)
    #plt.colorbar()
    #filename = 'newheatmap' +str(i).zfill(4) +'.png'
    #plt.savefig("output_images_test2/" + filename)   
    
    
    heatmap = np.clip(new_heat, 0, 255)
    new_labels = label(heatmap)
    
    #draw_img = draw_labeled_bboxes(np.copy(image), labels, bbox_queue[0:q])
    draw_img = draw_labeled_bboxes(np.copy(image), new_labels, new_box_list)
    draw_img = draw_img*255
    result_bgr = cv2.cvtColor(draw_img, cv2.COLOR_RGB2BGR)
    
    filename = 'result' +str(i).zfill(4) +'.png'
    cv2.imwrite(os.path.join(dirname, filename), result_bgr)
    
    
    
    
    ex_label.append(new_labels[0])
    ex_out_bounding_boxes.append(draw_img)
    
    
    
    i += 1





In [170]:
fig = plt.figure()
for i in range(0, 3):
   
    plt.subplot(6,2,i*2+1)
    plt.imshow(ex_images[i])
    plt.subplot(6,2,i*2)
    plt.imshow(ex_heatmap[i])
    #plt.imshow(ex_label[i])
    #plt.imshow(ex_bounding_boxes[i])
    
fig.tight_layout()
plt.savefig("examples/bboxes_and_heat.png")
#plt.savefig("examples/labels_map.png")
#plt.savefig("examples/output_bboxes.png")

(720, 1280, 3)

[1, 2, [...]]

ValueError: zero-dimensional arrays cannot be concatenated